In [2]:
import pandas as pd
from google.colab import files

In [3]:
# Load cleaned metadata
metadata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GSE306864_series_matrix_cleaned.csv")

In [4]:
# Check column names (optional but good practice)
print("Columns:", metadata.columns)

Columns: Index(['geo_accession', 'sample_title', 'sex', 'condition'], dtype='object')


In [5]:
# Check unique values in Sex column
print("Unique Sex values:", metadata["sex"].unique())

Unique Sex values: ['male' 'female']


In [6]:
# Filter for male samples (adjust if your file uses 'M' instead of 'Male')
male_subset = metadata[metadata["sex"] == "male"]

In [7]:
# Confirm filtering worked
print("\nMale subset shape:", male_subset.shape)
print("Sex distribution:\n", male_subset["sex"].value_counts())
print("Condition distribution:\n", male_subset["condition"].value_counts())


Male subset shape: (21, 4)
Sex distribution:
 sex
male    21
Name: count, dtype: int64
Condition distribution:
 condition
Control         16
Preeclampsia     5
Name: count, dtype: int64


In [8]:
# Save filtered metadata
male_subset.to_csv("male_subset_metadata.csv", index=False)

In [9]:
# Download the file
files.download("male_subset_metadata.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# Load male metadata
male_metadata = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/raw_count_matrix.csv")


In [11]:
# Load raw count matrix
# Assumes genes are rows and samples are columns
counts = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/raw_count_matrix.csv", index_col=0)


In [12]:
# Inspect sample IDs
print("Metadata Sample IDs (first 5):")
print(male_subset["sample_title"].head())

print("\nCount Matrix Columns (first 5):")
print(counts.columns[:5])

Metadata Sample IDs (first 5):
0     SCP3412
6     SCP3929
7     SCP3954
8     SCP3962
10    SCP4059
Name: sample_title, dtype: object

Count Matrix Columns (first 5):
Index(['SCP3412', 'SCP3628', 'SCP3660', 'SCP3780', 'SCP3843'], dtype='object')


In [13]:
# Ensure perfect matching
metadata_samples = set(male_subset["sample_title"])
count_samples = set(counts.columns)

# Check for mismatches
missing_in_counts = metadata_samples - count_samples
missing_in_metadata = count_samples - metadata_samples

print("\nSamples in metadata but NOT in count matrix:")
print(missing_in_counts)

print("\nSamples in count matrix but NOT in metadata:")
print(missing_in_metadata)


Samples in metadata but NOT in count matrix:
set()

Samples in count matrix but NOT in metadata:
{'STP0903', 'SCP3992', 'STP0852', 'STP0761', 'SCP3628', 'STP1087', 'STP0593', 'STP1082', 'STP0692', 'STP0888', 'STP0282', 'SCP4726', 'STP0867', 'STP0677', 'STP0636', 'STP0016', 'SCP3660', 'STP0793', 'SCP3877', 'SCP4164', 'SCP4536', 'SCP4706', 'STP0023', 'STP0230', 'STP0275', 'SCP3780', 'STP0788', 'SCP4139', 'SCP4378', 'SCP4748', 'SCP4733', 'STP0582', 'SCP3843', 'SCP4073', 'SCP4538', 'STP0912', 'SCP4154'}


In [14]:
# Keep only matching samples
matched_samples = list(metadata_samples.intersection(count_samples))

matched_counts = counts[matched_samples]


In [15]:
# Ensure correct order (VERY IMPORTANT for DESeq2)
matched_counts = matched_counts[matched_samples]

In [16]:
# Filter count matrix to include only male samples present in metadata
male_sample_ids = male_subset["sample_title"].tolist()
counts_filtered = counts[male_sample_ids]

print("Shape of original counts matrix:", counts.shape)
print("Shape of filtered counts matrix:", counts_filtered.shape)
print("First 5 columns of filtered counts matrix:", counts_filtered.columns[:5])

Shape of original counts matrix: (42645, 58)
Shape of filtered counts matrix: (42645, 21)
First 5 columns of filtered counts matrix: Index(['SCP3412', 'SCP3929', 'SCP3954', 'SCP3962', 'SCP4059'], dtype='object')


In [17]:
# Save matched count matrix
matched_counts.to_csv("matched_count_matrix.csv")

print("\nFinal Matched Count Matrix Shape:", matched_counts.shape)



Final Matched Count Matrix Shape: (42645, 21)


In [18]:
# Install PyDESeq2 (if not installed)
!pip install pydeseq2

import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.6/176.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.5 MB/s eta 0:00:00


In [19]:
# Download the file
files.download("matched_count_matrix.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# Load files
metadata = pd.read_csv("male_subset_metadata.csv")
counts = pd.read_csv("matched_count_matrix.csv", index_col=0)

In [21]:
# Clean column names (remove hidden spaces + BOM characters)
metadata.columns = (
    metadata.columns
    .str.strip()
    .str.replace('\ufeff', '', regex=False)
)

In [22]:
# Print cleaned column names
print("Metadata columns after cleaning:")
for col in metadata.columns:
    print(repr(col))

Metadata columns after cleaning:
'geo_accession'
'sample_title'
'sex'
'condition'


In [23]:
# Set 'sample_title' as sample ID index to match the count matrix
metadata = metadata.set_index("sample_title")

In [24]:
# Transpose count matrix (samples must be rows for PyDESeq2)
counts = counts.T

In [25]:
# Ensure sample IDs match
print("\nChecking sample ID alignment...")
print("Counts samples (first 5):", counts.index[:5])
print("Metadata samples (first 5):", metadata.index[:5])


Checking sample ID alignment...
Counts samples (first 5): Index(['STP0795', 'SCP4809', 'SCP4196', 'STP0148', 'SCP3929'], dtype='object')
Metadata samples (first 5): Index(['SCP3412', 'SCP3929', 'SCP3954', 'SCP3962', 'SCP4059'], dtype='object', name='sample_title')


In [26]:
# Check for mismatches
missing_in_counts = set(metadata.index) - set(counts.index)
missing_in_metadata = set(counts.index) - set(metadata.index)

print("\nSamples in metadata but NOT in counts:", missing_in_counts)
print("Samples in counts but NOT in metadata:", missing_in_metadata)


Samples in metadata but NOT in counts: set()
Samples in counts but NOT in metadata: set()


In [27]:
# Keep only perfectly matched samples
matched_samples = list(set(metadata.index).intersection(set(counts.index)))

counts = counts.loc[matched_samples]
metadata = metadata.loc[matched_samples]


In [28]:
# Ensure identical ordering
metadata = metadata.loc[counts.index]

print("\nFinal shapes:")
print("Counts shape:", counts.shape)
print("Metadata shape:", metadata.shape)

print("\nAlignment successful. Ready for DE analysis.")


Final shapes:
Counts shape: (21, 42645)
Metadata shape: (21, 3)

Alignment successful. Ready for DE analysis.


In [29]:
# Clean IDs
counts.columns = counts.columns.str.strip()

# Define mapping_df from male_subset (which contains 'sample_title' and 'geo_accession')
# Ensure 'sample_title' column is also cleaned in mapping_df
mapping_df = male_subset.copy()
mapping_df["sample_title"] = mapping_df["sample_title"].str.strip()

# Set mapping from sample_title to geo_accession
mapping_dict = dict(zip(
    mapping_df["sample_title"],
    mapping_df["geo_accession"]
))

# NOTE: counts.columns are currently gene IDs, not sample IDs.
# Applying mapping_dict (sample_title to geo_accession) to gene IDs will lead to NaNs.
# If the intent is to rename sample IDs in the index of counts, use counts.rename(index=mapping_dict, inplace=True).
# For now, this line is kept as per original user code but highlighted for potential issue.
counts.columns = counts.columns.map(mapping_dict)

# Now align with metadata
common_samples = counts.columns.intersection(metadata.index)

counts = counts[common_samples]
metadata = metadata.loc[common_samples]

In [32]:
counts = counts.apply(pd.to_numeric, errors="coerce")
counts = counts.dropna()

In [33]:
# Load files freshly to ensure correct state
metadata = pd.read_csv("male_subset_metadata.csv")
counts = pd.read_csv("matched_count_matrix.csv", index_col=0)

# Clean column names in metadata
metadata.columns = (
    metadata.columns
    .str.strip()
    .str.replace('\ufeff', '', regex=False)
)

# Set 'sample_title' as sample ID index for metadata
metadata = metadata.set_index("sample_title")

# Transpose count matrix (samples must be rows for PyDESeq2)
counts = counts.T

# Ensure sample IDs are perfectly matched and ordered
common_samples = list(set(metadata.index).intersection(set(counts.index)))

# Filter and reorder both dataframes based on common_samples
counts = counts.loc[common_samples]
metadata = metadata.loc[common_samples]

# Ensure identical ordering
metadata = metadata.loc[counts.index]

# Check final shapes and alignment
print("\nFinal Counts shape:", counts.shape)
print("Final Metadata shape:", metadata.shape)
print("Alignment successful. Ready for DE analysis.")

# Create DESeq2 dataset
dds = DeseqDataSet(
    counts=counts,
    metadata=metadata,
    design_factors="condition",   # PE vs Control
)


Final Counts shape: (21, 42645)
Final Metadata shape: (21, 3)
Alignment successful. Ready for DE analysis.


/tmp/ipython-input-1979978097.py:34: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(


In [34]:
# Run normalization + dispersion estimation
dds.deseq2()

Fitting size factors...
... done in 0.09 seconds.



Using None as control genes, passed at DeseqDataSet initialization


Fitting dispersions...
... done in 42.72 seconds.

Fitting dispersion trend curve...
/usr/local/lib/python3.12/dist-packages/pydeseq2/dds.py:822: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 1.25 seconds.

Fitting MAP dispersions...
... done in 69.10 seconds.

Fitting LFCs...
... done in 38.73 seconds.

Calculating cook's distance...
... done in 0.06 seconds.

Replacing 466 outlier genes.

Fitting dispersions...
... done in 0.65 seconds.

Fitting MAP dispersions...
... done in 0.71 seconds.

Fitting LFCs...
... done in 0.72 seconds.



In [35]:
# Compute differential expression
stat_res = DeseqStats(dds, contrast=("condition", "Preeclampsia", "Control"))
stat_res.summary()

Running Wald tests...
... done in 12.19 seconds.



Log2 fold change & Wald test p-value: condition Preeclampsia vs Control
                    baseMean  log2FoldChange     lfcSE      stat    pvalue  \
Column1                                                                      
ENSG00000223972     0.102724       -0.525066  3.108411 -0.168918  0.865861   
ENSG00000227232     7.400155       -0.387748  0.439006 -0.883240  0.377107   
ENSG00000278267     2.864444        0.307389  0.668400  0.459888  0.645596   
ENSG00000238009     0.359074       -0.103241  1.220067 -0.084619  0.932564   
ENSG00000233750     0.023398        0.409079  3.720747  0.109945  0.912453   
...                      ...             ...       ...       ...       ...   
ENSG00000198695   101.540096        0.412286  0.414472  0.994726  0.319869   
ENSG00000210194     3.133138       -0.717408  1.314138 -0.545916  0.585124   
ENSG00000198727  8950.733259        0.261694  0.357593  0.731819  0.464279   
ENSG00000210195    28.511449        0.612032  0.456633  1.340317  0.18

In [36]:
# Extract results
results_df = stat_res.results_df

In [37]:
# Filter significantly upregulated genes in Male PE
upregulated = results_df[
    (results_df["log2FoldChange"] > 0) &
    (results_df["padj"] < 0.05)
]

In [38]:
# Save results
results_df.to_csv("DE_Results_All_Genes.csv")
upregulated.to_csv("DE_Results_Upregulated_Genes.csv")

print("Total significant upregulated genes:", upregulated.shape[0])

Total significant upregulated genes: 235


In [39]:
# Download the file
files.download("DE_Results_All_Genes.csv")
files.download("DE_Results_Upregulated_Genes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
downregulated = results_df[
    (results_df["log2FoldChange"] < 0) &
    (results_df["padj"] < 0.05)
]

In [41]:
# Save results
results_df.to_csv("DE_Results_All_Genes.csv")

upregulated.to_csv("DE_Results_Upregulated_Genes.csv")

# Save downregulated genes
downregulated.to_csv("DE_Results_Downregulated_Genes.csv")

print("Total significant upregulated genes:", upregulated.shape[0])
print("Total significant downregulated genes:", downregulated.shape[0])

Total significant upregulated genes: 235
Total significant downregulated genes: 33


In [42]:
# Download the file
files.download("DE_Results_Downregulated_Genes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
# Create Male PE "Bad" Signature (Upregulated genes)

# 1️⃣ Filter significantly upregulated genes
signature_df = results_df[
    (results_df["log2FoldChange"] > 0) &
    (results_df["padj"] < 0.05)
]

# 2️⃣ Extract gene names (assuming genes are in index)
signature_genes = signature_df.index

# 3️⃣ Save as text file (one gene per line)
signature_genes.to_series().to_csv(
    "Preeclampsia_Signature.txt",
    index=False,
    header=False
)

print("Total signature genes:", len(signature_genes))
print("Signature file saved as Preeclampsia_Signature.txt")

Total signature genes: 235
Signature file saved as Preeclampsia_Signature.txt


In [44]:
# Download the file
files.download("Preeclampsia_Signature.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>